# Проект: Защита персональных данных клиентов

# Задача
Мне нужно защитить данные клиентов страховой компании. В ходе проекта мною будет разработан такой метод преобразования данных, что по ним будет сложно восстановить персональную информацию.

# Условие
Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

# Стек
Pandas, Numpy, Sklearn

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [2]:
state = 12345

In [3]:
try:
    data = pd.read_csv('/datasets/insurance.csv')
except: 
    data = pd.read_csv("C:/Users/gorok/Desktop/datasets/insurance.csv")

In [4]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

In [6]:
features = data.drop('Страховые выплаты', axis=1).values
target = data['Страховые выплаты'].values

In [7]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.15, random_state=state)

In [8]:
model = LinearRegression()
model.fit(features_train, target_train)
res = model.predict(features_test)
r2_score(target_test, res)

0.41021696040877087

Создали обратимую матрицу, ее определитель(детерминатор) не равен нулю: проверил в онлайн калькуляторе, можно было бы проверить при помощи linalg inv

In [9]:
inv_matrix = [[57,74,7,79],[68,94,10,40],[72,54,27,54],[24,8,98,43]]

In [10]:
inv_matrix = np.array(inv_matrix)
inv_matrix

array([[57, 74,  7, 79],
       [68, 94, 10, 40],
       [72, 54, 27, 54],
       [24,  8, 98, 43]])

In [11]:
np.linalg.det(inv_matrix)

11913570.000000013

In [12]:
features_train, features_test, target_train, target_test = train_test_split(
    features@inv_matrix, target, test_size=0.15, random_state=state)

In [13]:
model = LinearRegression()
model.fit(features_train, target_train)
res = model.predict(features_test)
r2_score(target_test, res)

0.4102169604087207

**Ответ:** Результат не изменился 

**Обоснование:** $$
$$
w = (X^T X)^{-1} X^T y
$$

$$ 
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
где P - обратимая матрица
$$ 
w' = (P^T (X^T X)P)^{-1} P^T X^T y
$$
В минус -1 степень возводятся квадратные матрицы
$$ 
w' = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$
$$
PP^{-1}=E
$$
Подсавляем самое первое выражение
$$
w' = P^{-1} w $$

## Алгоритм преобразования

**Алгоритм**

Мой алгоритм заключается в последовательном умножении заранее выбранной матрицы на матрицу признак и траспонировании матриц и результатов
1. Транспонируем выбранную матрицу 
2. Транспонируем матрицу признаков, чтобы размерность матриц совпала
3. Перемножаем получившиеся матрицы в соответсвующем порядке
4. Транспонируем результат, чтобы вернуть матрицу в исходное состояние и далее умножить 
5. Умножаем матрицу из 4 пункта на матрицу, которую мы транспонировали в пункте 1
** Выбранная матрица должна быть обратимой

Результат: Качество линейной регрессии не изменилось, исходные данные изменены до неузнаваемости 

Основываясь на ранее полученных результатах, получаем обоснование правдоподобности нашего алгоритма. Наша матрица признаков несколько раз умножается на обратимую матрицу, что в свою очередь означает:

$$
w = (X^T X)^{-1} X^T y
$$

$$ 
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
где P - обратимая матрица
$$ 
w' = (P^T (X^T X)P)^{-1} P^T X^T y
$$
В минус -1 степень возводятся квадратные матрицы
$$ 
w' = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$
$$
PP^{-1}=E
$$
Подсавляем самое первое выражение
$$
w' = P^{-1} w $$

**Обоснование:**

Используемые свойства:
$$
(AB)^T=B^T A^T
$$
$$
(AB)^{-1} = B^{-1} A^{-1}
$$
$$
A A^{-1} = A^{-1} A = E
$$
$$
AE = EA = A
$$
Доказательство:
$$
a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w'
$$
\
Требуется доказать, что предсказания не изменятся, имеем  $a =  Xw$,   $a' = X'w'$
\
\
$$
    w = (X^T X)^{-1} X^T y
$$
\
$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} P^T X^T y
$$



## Проверка алгоритма

### Исходная модель 

In [14]:
features = data.drop('Страховые выплаты', axis=1).values
target = data['Страховые выплаты'].values

In [15]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.15, random_state=state)

In [16]:
model = LinearRegression()
model.fit(features_train, target_train)
res = model.predict(features_test)
r2_score(target_test, res)

0.41021696040877087

### Модель с использованием алгоритма скрытия персональных данных

In [17]:
our_inv_matrix = [[17,20,4,11],[4,9,11,23],[22,40,49,33],[34,16,25,43]]
our_inv_matrix = np.array(our_inv_matrix)

In [18]:
np.linalg.det(our_inv_matrix)

364429.9999999998

In [19]:
new_features = (our_inv_matrix.T @ features.T).T @ our_inv_matrix
new_target = data['Страховые выплаты'].values

In [20]:
features_train_A, features_test_A, target_train_A, target_test_A = train_test_split(
new_features, new_target, random_state=state, test_size=0.15)

In [21]:
model = LinearRegression()
model.fit(features_train_A, target_train_A)
res = model.predict(features_test_A)
r2_score(target_test_A, res)

0.41021696040866973

### Обратный алгоритм

In [22]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [23]:
new_features

array([[1.35656133e+08, 1.63127897e+08, 1.86252160e+08, 2.08291191e+08],
       [1.03946542e+08, 1.24991086e+08, 1.42711146e+08, 1.59601154e+08],
       [5.74467120e+07, 6.90761010e+07, 7.88696410e+07, 8.82044870e+07],
       ...,
       [9.27104680e+07, 1.11487604e+08, 1.27290404e+08, 1.42351092e+08],
       [8.94354090e+07, 1.07547310e+08, 1.22791433e+08, 1.37321366e+08],
       [1.11035469e+08, 1.33523300e+08, 1.52450183e+08, 1.70488352e+08]])

In [24]:
new_features = (np.linalg.inv(our_inv_matrix.T) @ new_features.T).T @ np.linalg.inv(our_inv_matrix)
new_features

array([[ 1.00000000e+00,  4.10000000e+01,  4.96000000e+04,
         1.00000000e+00],
       [ 5.29401312e-11,  4.60000000e+01,  3.80000000e+04,
         1.00000000e+00],
       [ 2.84255929e-11,  2.90000000e+01,  2.10000000e+04,
        -2.20537882e-11],
       ...,
       [ 3.38470323e-12,  2.00000000e+01,  3.39000000e+04,
         2.00000000e+00],
       [ 1.00000000e+00,  2.20000000e+01,  3.27000000e+04,
         3.00000000e+00],
       [ 1.00000000e+00,  2.80000000e+01,  4.06000000e+04,
         1.00000000e+00]])

## Вывод 

В данном проекте, мною был представлен метод сокрытия личных данных путем алгоритма с использоваем матриц и матричных операций. Был предоставлен обратный алгоритм. Качество линейной регрессии не искажается после преобразования

Заказчик может отправлять данные заранее их преобразовав. Это может понадобиться при передаче данных между филиалами компании. Ко всему прочему, так как качество линейной регрессии не пострадало, заказчик может давать такие преобразованные данных специалистам по машинному обучени, если в этом есть необходимость 